# Multi-Dataset ImageNet Training with ResNet50 - Modular Version

This notebook demonstrates training ResNet50 on multiple ImageNet variants using a modular approach. The code supports experiments across different datasets for comprehensive analysis.

## Supported Datasets:
- **ImageNette**: 10 classes, 224x224 images (fastest for experiments)
- **Tiny ImageNet**: 200 classes, 64x64 images (medium complexity)
- **ImageNet Mini**: 1000 classes, 224x224 images (subset of full ImageNet)
- **Full ImageNet**: 1000 classes, 224x224 images (full dataset)

## Features:
- **Modular Design**: Separate modules for configuration, data loading, models, and training utilities
- **Multi-Dataset Support**: Easy switching between different ImageNet variants
- **Dataset-Specific Training**: Optimized hyperparameters for each dataset
- **Comprehensive Metrics**: Tracks training and validation metrics
- **Model Saving**: Automatic model checkpointing with dataset-specific naming


In [1]:
# 1️⃣ Mount Google Drive
drive.mount('/content/gdrive')

# 2️⃣ Define your Drive path (folder + file)
drive_folder = '/content/gdrive/My Drive/models'
os.makedirs(drive_folder, exist_ok=True)
file_path = os.path.join(drive_folder, 'dummy_direct_save.txt')

# 3️⃣ Write directly to Google Drive
with open(file_path, 'w') as f:
    f.write("✅ This file was written directly to Google Drive.\n")

print(f"📂 File written directly to: {file_path}")

# 4️⃣ Verify it exists
!ls -lh "/content/gdrive/My Drive/models"

SyntaxError: invalid character '️' (U+FE0F) (ipython-input-3507605965.py, line 1)

In [ ]:
# Install required packages
%pip install torchsummary albumentations


In [3]:
# Step 1: Clone the repo
!rmdir ERAv4S9
!git clone https://github.com/nitin-vig/ERAv4S9.git

# Step 2: Move into the repo folder
%cd ERAv4S9

# Step 3: (Optional) List files to verify
!ls -l

rmdir: failed to remove 'ERAv4S9': No such file or directory
Cloning into 'ERAv4S9'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 160 (delta 84), reused 97 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (160/160), 3.66 MiB | 7.40 MiB/s, done.
Resolving deltas: 100% (84/84), done.
/content/ERAv4S9
total 2360
-rw-r--r-- 1 root root   13109 Oct 28 00:19 config.py
-rw-r--r-- 1 root root   25618 Oct 28 00:19 dataset_loader.py
-rw-r--r-- 1 root root 2312132 Oct 28 00:19 ImageNet_Experiment_Resnet_50.ipynb
-rw-r--r-- 1 root root    7655 Oct 28 00:19 models.py
-rw-r--r-- 1 root root    4365 Oct 28 00:19 NOTEBOOK_MULTI_STAGE_GUIDE.md
-rw-r--r-- 1 root root    3521 Oct 28 00:19 progressive_transfer_learning_example.py
-rw-r--r-- 1 root root   10776 Oct 28 00:19 README.md
-rw-r--r-- 1 root root     648 Oct 28 00:19 requirements.txt
-rw-r--r-- 1 root root   20006 Oct 28 00:19 t

In [4]:
# Import all necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import os
from google.colab import drive

# Import our modular components
from config import Config
from dataset_loader import get_data_loaders, visualize_samples
from models import get_model, count_parameters, get_model_summary, save_model
from training_utils import train_model, train_model_with_transfer, evaluate_model, MetricsTracker, verify_saved_files

print(Config.__dict__)

print("All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


KeyboardInterrupt: 

## Configuration

Let's configure our training parameters. You can easily switch between datasets and modify training parameters here.


In [ ]:
# Configuration setup for multi-dataset experiments
# You can easily switch between datasets by changing DATASET_NAME

# Dataset configuration
DATASET_NAME = "imagenette"  # Options: "imagenette", "tiny_imagenet", "imagenet_mini", "imagenet"
USE_PRETRAINED = False  # Custom implementation without pretrained weights

# Update configuration for the selected dataset
Config.update_for_dataset(DATASET_NAME)

print("Configuration updated!")
print(f"Dataset: {DATASET_NAME}")
print(f"Image size: {Config.IMAGE_SIZE}")
print(f"Number of classes: {Config.NUM_CLASSES}")
print(f"Batch size: {Config.BATCH_SIZE}")
print(f"Epochs: {Config.NUM_EPOCHS}")
print(f"Learning rate: {Config.LEARNING_RATE}")
print(f"Use pretrained: {USE_PRETRAINED}")

# Display dataset-specific training parameters
dataset_config = Config.get_dataset_config(DATASET_NAME)
print(f"\nDataset-specific parameters:")
print(f"Optimizer: {dataset_config['optimizer']}")
print(f"Scheduler: {dataset_config['scheduler']}")


## Environment Setup

Setup the environment and check GPU availability. For full ImageNet training, you'll need significant computational resources.


In [ ]:
# Setup environment
def setup_environment():
    """Setup environment for ImageNet training"""
    print("Setting up environment for ImageNet training...")

    # Create necessary directories
    os.makedirs(Config.DATA_ROOT, exist_ok=True)
    os.makedirs(Config.SAVE_MODEL_PATH, exist_ok=True)

    print("Environment setup complete!")

def check_gpu_availability():
    """Check GPU availability and setup device"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
        print("Warning: Full ImageNet training requires significant GPU memory!")
    else:
        print("Warning: CPU training will be very slow for ImageNet!")

    return device

# Run setup
setup_environment()
device = check_gpu_availability()

# Set random seed for reproducibility
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)


In [ ]:
# 🧩 Colab Dataset Setup Cell
# Copy-paste this cell into your Colab notebook

import os
import subprocess

# Install required packages (if not already installed)
!pip install -q torch torchvision albumentations tqdm requests

DATA_DIR = "/content/data"
IMAGENETTE_DIR = os.path.join(DATA_DIR, "imagenette2")
TINY_IMAGENET_DIR = os.path.join(DATA_DIR, "tiny-imagenet-200")

os.makedirs(DATA_DIR, exist_ok=True)

def run_cmd(cmd):
    """Helper to run shell commands cleanly."""
    result = subprocess.run(cmd, shell=True, check=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        print(f"⚠️ Warning: {cmd} failed with error:\n{result.stderr}")
    return result

# ------------------------------
# 🔹 Download ImageNette
# ------------------------------
if not os.path.exists(IMAGENETTE_DIR) or len(os.listdir(IMAGENETTE_DIR)) == 0:
    print("🔄 Downloading ImageNette...")
    run_cmd("wget -q https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz")
    run_cmd("tar -xzf imagenette2.tgz")
    run_cmd(f"mv imagenette2 {DATA_DIR}/")
    run_cmd("rm imagenette2.tgz")
    print("✅ ImageNette downloaded!")
else:
    print(f"✅ Skipping ImageNette — already exists at {IMAGENETTE_DIR}")

# ------------------------------
# 🔹 Download Tiny ImageNet
# ------------------------------
if not os.path.exists(TINY_IMAGENET_DIR) or len(os.listdir(TINY_IMAGENET_DIR)) == 0:
    print("🔄 Downloading Tiny ImageNet...")
    run_cmd("wget -q http://cs231n.stanford.edu/tiny-imagenet-200.zip")
    run_cmd("unzip -q tiny-imagenet-200.zip")
    run_cmd(f"mv tiny-imagenet-200 {DATA_DIR}/")
    run_cmd("rm tiny-imagenet-200.zip")
    print("✅ Tiny ImageNet downloaded!")
else:
    print(f"✅ Skipping Tiny ImageNet — already exists at {TINY_IMAGENET_DIR}")

# ------------------------------
# 🔹 Verify
# ------------------------------
print("\n📁 Dataset verification:")
!ls -la {DATA_DIR}
!du -sh {DATA_DIR}/* | sort -h

print("\n🎉 Datasets ready! You can now use:")
print("from dataloader import get_data_loaders")
print("train_loader, test_loader = get_data_loaders('imagenette')")


## Data Loading

Load the dataset and visualize some sample images.


In [ ]:
# Load dataset
print("Loading dataset...")
train_loader, test_loader = get_data_loaders(DATASET_NAME)

# Visualize some samples
print("\nVisualizing sample images...")
visualize_samples(train_loader, num_samples=12)


## Model Creation

Create the ResNet50 model and display its architecture.


In [ ]:
# Create model
print(f"Creating {Config.MODEL_NAME} model...")
model = get_model(
    model_name=Config.MODEL_NAME,
    dataset_name=DATASET_NAME,
    pretrained=USE_PRETRAINED
)

# Move model to device
model = model.to(device)

# Print model info
print(f"Model parameters: {count_parameters(model):,}")

# Get model summary
dataset_config = Config.get_dataset_config(DATASET_NAME)
input_size = (3, dataset_config["image_size"], dataset_config["image_size"])
print(f"\nModel summary (input size: {input_size}):")
get_model_summary(model, input_size=input_size)


## Training

Train the model using our modular training utilities.


In [ ]:
# # Train model
# stage_config = Config.STAGES[DATASET_NAME]
# print(stage_config)
# NUM_EPOCHS = stage_config["epochs"]
# BATCH_SIZE = stage_config["batch_size"]
# LEARNING_RATE = stage_config["lr"]
# WEIGHT_DECAY = stage_config["weight_decay"]

# print(f"Starting training for {NUM_EPOCHS} epochs...")
# print(f"Batch size: {BATCH_SIZE}")
# print(f"Learning rate: {LEARNING_RATE}")
# print(f"Weight decay: {WEIGHT_DECAY}")

# metrics_tracker = train_model(model, train_loader, test_loader, device, Config)


## Results Visualization

Plot the training metrics and evaluate the final model.


In [ ]:
# # Plot training metrics
# print("Plotting training metrics...")
# metrics_tracker.plot_metrics(save_path=f"{Config.SAVE_MODEL_PATH}/training_metrics.png")

# # Final evaluation
# print("\nFinal evaluation...")
# test_loss, test_acc, test_top5_acc = evaluate_model(model, test_loader, device)

# print(f"\nTraining completed!")
# print(f"Final Test Accuracy: {test_acc:.2f}%")
# print(f"Final Top-5 Accuracy: {test_top5_acc:.2f}%")


## Model Saving

Save the trained model to local storage and Google Drive.


In [ ]:
# # Save final model
# final_model_path = f"{Config.SAVE_MODEL_PATH}/final_model.pth"
# save_model(model, final_model_path, epoch=NUM_EPOCHS, loss=test_loss)

# # Save to Google Drive if mounted
# if Config.MOUNT_DRIVE:
#     drive_model_path = f"{Config.DRIVE_MODEL_PATH}/final_model.pth"
#     save_model(model, drive_model_path, epoch=NUM_EPOCHS, loss=test_loss)
#     print(f"Model also saved to Google Drive: {drive_model_path}")

# print("Model saving completed!")


## Progressive Transfer Learning (Multi-Stage Training)

Run multiple stages in sequence with automatic weight transfer from each stage to the next.


In [ ]:
"""
PROGRESSIVE TRANSFER LEARNING - RUN SELECTED STAGES
===================================================
Train multiple stages in sequence with automatic weight transfer.
"""

import json
from datetime import datetime

# ⚙️ CONFIGURATION - Choose which stages to run
# ================================================
STAGES_TO_RUN = ["imagenette", "tiny_imagenet"]  # Modify this list
# Options: "imagenette", "tiny_imagenet", "imagenet_mini", "imagenet"
# Order matters! Stages run in sequence with weight transfer

USE_PRETRAINED_FOR_FIRST_STAGE = False  # Start with ImageNet pretrained weights?
SAVE_RESULTS_AT_EACH_STAGE = True  # Save models and metrics at each stage?

# Results storage
all_stage_results = {}
pretrained_weights_path = None

# Verify save path exists (create if needed)
print("🚀 Progressive Transfer Learning Training")
print(f"Stages to run: {STAGES_TO_RUN}")
print(f"Start with pretrained: {USE_PRETRAINED_FOR_FIRST_STAGE}")
print(f"Save at each stage: {SAVE_RESULTS_AT_EACH_STAGE}")
print("="*80)

# Check if save path exists
save_path = Config.SAVE_MODEL_PATH
if not os.path.exists(save_path):
    print(f"\n⚠️ Save path doesn't exist: {save_path}")
    print(f"Creating directory...")
    os.makedirs(save_path, exist_ok=True)
    print(f"✅ Created: {save_path}")
else:
    print(f"\n📁 Models will be saved to: {save_path}")
    print(f"   (Directory exists)")

# Verify Google Drive is mounted (if using /gdrive)
if save_path.startswith('/content/gdrive'):
    gdrive_root = '/content/gdrive'
    if not os.path.exists(gdrive_root):
        print(f"\n❌ ERROR: Google Drive not mounted!")
        print(f"Please run the 'Mount Google Drive' cell (Cell 11) first.")
        print(f"After mounting, Google Drive will be available at: /content/gdrive")
    else:
        print(f"✅ Google Drive is mounted at: {gdrive_root}")

print("="*80)

for i, stage_name in enumerate(STAGES_TO_RUN, 1):
    print(f"\n{'='*80}")
    print(f"STAGE {i}/{len(STAGES_TO_RUN)}: {stage_name.upper()}")
    print(f"{'='*80}")

    # Update configuration for this stage
    Config.update_for_dataset(stage_name)

    # Get stage configuration
    stage_config = Config.STAGES[stage_name]
    print(f"Classes: {stage_config['classes']}")
    print(f"Image size: {stage_config['image_size']}")
    print(f"Epochs: {stage_config['epochs']}")
    print(f"Batch size: {stage_config['batch_size']}")
    print(f"LR: {stage_config['lr']}")
    print(f"Previous weights: {pretrained_weights_path or 'None (fresh start)'}")

    # Load dataset
    print(f"\n📦 Loading {stage_name} dataset...")
    train_loader, test_loader = get_data_loaders(stage_name)

    # Create model
    print(f"\n🧠 Creating model for {stage_name}...")
    use_pretrained = USE_PRETRAINED_FOR_FIRST_STAGE if (i == 1 and not pretrained_weights_path) else False
    model = get_model(
        model_name=Config.MODEL_NAME,
        dataset_name=stage_name,
        pretrained=use_pretrained
    )
    model = model.to(device)
    print(f"Model created: {count_parameters(model):,} parameters")

    # Determine next stage name for weight saving
    next_stage = STAGES_TO_RUN[i] if i < len(STAGES_TO_RUN) else None

    # Train with transfer learning
    print(f"\n🏋️ Training {stage_name}...")
    metrics_tracker, final_weights = train_model_with_transfer(
        model=model,
        train_loader=train_loader,
        test_loader=test_loader,
        device=device,
        config=Config,
        pretrained_weights_path=pretrained_weights_path,
        next_stage_name=next_stage
    )

    # Save results if requested
    if SAVE_RESULTS_AT_EACH_STAGE:
        stage_save_dir = f"{Config.SAVE_MODEL_PATH}/{stage_name}_stage_{i}"
        os.makedirs(stage_save_dir, exist_ok=True)

        # Save final model
        final_model_path = f"{stage_save_dir}/final_model.pth"
        torch.save(model.state_dict(), final_model_path)
        print(f"✅ Saved model: {final_model_path}")

        # Save metrics
        metrics_data = {
            "stage": stage_name,
            "stage_number": i,
            "final_train_loss": metrics_tracker.train_losses[-1] if metrics_tracker.train_losses else None,
            "final_train_acc": metrics_tracker.train_acc[-1] if metrics_tracker.train_acc else None,
            "final_test_loss": metrics_tracker.test_losses[-1] if metrics_tracker.test_losses else None,
            "final_test_acc": metrics_tracker.test_acc[-1] if metrics_tracker.test_acc else None,
            "config": stage_config
        }

        metrics_path = f"{stage_save_dir}/metrics.json"
        with open(metrics_path, 'w') as f:
            json.dump(metrics_data, f, indent=2)
        print(f"✅ Saved metrics: {metrics_path}")

        # Save metrics plot (graph with training curves)
        plot_path = f"{stage_save_dir}/training_metrics.png"
        metrics_tracker.plot_metrics(save_path=plot_path)
        print(f"✅ Saved training graphs: {plot_path}")

        all_stage_results[stage_name] = {
            "metrics": metrics_data,
            "model_path": final_model_path,
            "plot_path": plot_path
        }

        # Set weights path for next stage
    if next_stage:
        pretrained_weights_path = f"{Config.SAVE_MODEL_PATH}/weights_for_{next_stage}.pth"

    # Free GPU memory before moving to next stage
    if torch.cuda.is_available():
        del model  # Delete model to free memory
        del metrics_tracker  # Delete metrics tracker
        torch.cuda.empty_cache()  # Clear CUDA cache
        print("🧹 GPU memory cleared for next stage")

    print(f"\n✅ Stage {i} completed!")
    print("-"*80)

# Final summary
print("\n" + "="*80)
print("🎉 ALL STAGES COMPLETE!")
print("="*80)
if SAVE_RESULTS_AT_EACH_STAGE:
    print("\n📊 Results Summary:")
    print(f"{'Stage':<20} {'Test Accuracy':<15} {'Model Path':<60}")
    print("-"*95)
    for stage_name, result in all_stage_results.items():
        final_test_acc = result['metrics'].get('final_test_acc', 'N/A')
        model_path = result.get('model_path', 'N/A')
        if isinstance(final_test_acc, (int, float)):
            print(f"{stage_name:<20} {final_test_acc:<15.2f} {model_path}")
        else:
            print(f"{stage_name:<20} {str(final_test_acc):<15} {model_path}")

    print(f"\n{'='*80}")
    print(f"📁 Base Save Directory: {Config.SAVE_MODEL_PATH}")
    if Config.SAVE_MODEL_PATH.startswith('/gdrive'):
        print(f"📂 Location: Google Drive (My Drive/models)")
    print(f"\n📦 For each stage, you'll find:")
    print(f"  • final_model.pth - Trained model weights")
    print(f"  • metrics.json - Training metrics")
    print(f"  • training_metrics.png - Training curves graph")

    # Verify files were actually saved
    print(f"\n{'='*80}")
    print("🔍 Verifying saved files:")
    verify_saved_files(Config.SAVE_MODEL_PATH)


In [ ]:

drive_model_path = f"{Config.DRIVE_MODEL_PATH}/final_model.pth"
save_model(model, drive_model_path, epoch=30, loss=0)
print(f"Model also saved to Google Drive: {drive_model_path}")
